In [0]:
with cte_data (shapes, regions) as
(
select transform( slice(split(value, '\n\n'), 1, size(split(value, '\n\n')) - 1)
                , s -> flatten(transform( split(split_part(s, ':\n', 2), '\n')
                                        , (t, i) -> transform( split(t, '')
                                                             , (u,j) -> struct(j-1 as x, i-1 as y, (u = '#')::tinyint as v))))
                )
     , transform( split(element_at(split(value, '\n\n'), -1), '\n')
                , r -> struct( split_part(split_part(r, ': ', 1), 'x', 1)::int as x
                             , split_part(split_part(r, ': ', 1), 'x', 2)::int as y
                             , transform(split(split_part(r, ': ', 2), ' '), t -> t::int) as quantity
                             ) )
  from read_files('/Volumes/waldar/fabien/aoc2025/aoc2025_12_input.txt', format => 'text', wholeText => true) as t (value)
)
  ,  cte_prep (minimum_area) as
(
select aggregate( transform(arrays_zip(transform(shapes, s -> size(filter(s, f -> f.v = 1))), region.quantity), u -> (u.`0` * u.quantity)::int)
                , 0
                , (acc, x) -> acc + x
                , acc -> acc < region.x * region.y
                )
  from cte_data
  join lateral explode(regions) as t (region)
)
select count(*) as part1
  from cte_prep
 where minimum_area;
